In [2]:
import os
import pandas as pd
import numpy as np
import requests
import re, os, math
import collections
import pickle
from datetime import datetime
import time
import json

from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta

from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import sys
#sys.path.insert(0, "Users\medici\Desktop\clustering4docs-master\soyclustering")
#from soyclustering import SphericalKMeans

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize']= 12,6
plt.rcParams['axes.unicode_minus'] = False

In [3]:
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import *

In [4]:
import warnings
warnings.filterwarnings('ignore')

# 1. NEWS Title 대상 감성분석
- 비지도학습 : 군산대 감성사전 + 부동산 도메인 감성사전 적용

## 1) (1차) 긍부정 라벨링 감성분석

In [6]:
def score(x) :
    if x == '부정' :
        return -1
    elif x == '긍정' :
        return 1
    else :
        return 0
    
# title topic top1000개 감성라벨링한 파일
df_words = pd.read_csv('../DATA/news_title_top1000_pos_neg.csv', encoding = 'euc-kr')
df_words = df_words.fillna(0)

df_words['score'] = [score(x) for x in df_words['통일']]
df_words.head(20)

,Unnamed: 0,0,1,sj_긍부정,yj_긍부정,통일,score
0,0,거래,16134,0,0,0,0
1,1,아파트,14114,0,0,0,0
2,2,부동산,13368,0,0,0,0
3,3,전용,13144,0,0,0,0
4,4,서울특별시,9921,0,0,0,0
5,5,서울,7857,0,0,0,0
6,6,분양,7305,0,0,0,0
7,7,올라,7178,긍정,긍정,긍정,1
8,8,집값,6490,0,0,0,0
9,9,주택,5633,0,0,0,0


In [7]:
senti_dict_domain = {k : v for k, v in zip(df_words['0'], df_words['score'])}

## 2) (2차) [군산대 감성사전] 감성분석

In [9]:
#군산대학교 감성사전 : http://dilab.kunsan.ac.kr/knusl.html 
with open("SentiWord_info.json", encoding='utf-8-sig') as f :
    sentiwords = json.load(f)
    f.close()

In [17]:
sentiword_dict_K = {x['word'] : (int(x['polarity']) * 0.25) for x in sentiwords}

In [18]:
# 1차, 2차 감성분석 결합
sentiword_dict = dict(senti_dict_domain.items( ) | sentiword_dict_K.items( ) )   

## 3) [OpenKoreanText] 부동산 도메인 감성분석

In [19]:
from konlpy.tag import Okt
t = Okt()

def senti_score(sent) :
    tokens = t.morphs(sent)
    scores = [sentiword_dict.get(x, 0) for x in tokens]
    #print(tokens)
    #print(scores)
    return sum(scores)

- train

In [10]:
df_train = pd.read_pickle('../DATA/df_train.pkl')

In [22]:
title_senti = [(x, senti_score(x)) for x in df_train['title']]

- test

## 4) 결과확인

In [25]:
# 감성지수 내림차순 정렬
sorted(title_senti, key = lambda x : -x[1])[:5]

[('초저금리 시대에 매월 안정적인 수익과 환금성이 높은 수익형 오피스텔 인기 급증', 5.25),
 ('초저금리 시대에 매월 안정적인 수익과 환금성이 높은 수익형 오피스텔 인기 급증!', 5.25),
 ('낙찰가율 70%벽 돌파 토지 경매시장 고공행진 재개발 재건축도 꿈틀', 5),
 ("'재건축 연한 단축'호재에 1980년대 아파트 인기 낙찰가율 급등", 5),
 ("서울 도심 재개발 재건축 아파트에 관심 몰려 '북한산 더샵’ 분양 인기 상승", 5)]

In [27]:
# 감성지수 오름차순 정렬
sorted(title_senti, key = lambda x : x[1])[:5]

[('탈세 편법대출 난무 9억 이상 고가주택 불법거래 811건', -6),
 ("[정부 부동산 규제 가시화] 부산 부동산, 금리 인상 대출 규제 투기 지구 '3각 파도'에 기 꺾일까", -5.5),
 ('금리인상 악재 현실화된 부동산시장 하락세 심화 우려', -5.5),
 ("'풍선효과' 우려없나...고가 다주택자 대출규제 없어 한계도", -5.25),
 ('대출규제로 거래심리 ‘꽁꽁’ 신도시 분양권 웃돈 반토막', -5)]